<a href="https://colab.research.google.com/github/rockyistt/gis-code-ai/blob/main/notebooks/Train_GIS_Model_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Disable optional backends to prevent import conflicts (PyTorch-only training)
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
# Reduce CUDA fragmentation for large models
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print("✅ Env set: disabled TF/FLAX/TORCHVISION for transformers.")
print("✅ CUDA alloc conf: expandable_segments enabled.")

✅ Env set: disabled TF/FLAX/TORCHVISION for transformers.
✅ CUDA alloc conf: expandable_segments enabled.


# 🚀 GIS代码生成模型训练 - Google Colab (CodeLlama)

本Notebook在Google Colab上训练GIS代码生成模型（**文件级 + CodeLlama**）

**文件级训练** - 模型学习生成完整的工作流而不是单个步骤
- 输入：用户的高层指令（英语/荷兰语，如："Create MS and HS cable objects"）
- 输出：完整的工作流JSON代码（包含所有操作步骤）
- 优势：一次推理生成整个测试脚本
- **模型**：CodeLlama-7B-Instruct（专为代码生成优化）

**使用前准备：**
1. 运行环境：`Runtime > Change runtime type > T4 GPU`（免费）或 `A100 GPU`（Colab Pro）
2. 数据准备：确保已生成训练数据文件
3. 预计时间：4-6小时（T4）/ 1-2小时（A100）

---

## 📋 步骤1：环境设置

In [8]:
# 检查GPU
!nvidia-smi

Tue Jan 20 09:42:50 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
# 安装依赖（约3-5分钟）
print("📦 Installing dependencies...")

# 先锁定关键基础包（避免自动升级）
!pip install -q torch==2.9.0 --no-deps
!pip install -q fsspec==2024.3.1
!pip install -q numpy==2.0.2 --no-deps

# 安装主要训练库（指定兼容版本）
!pip install -q transformers==4.46.0
!pip install -q peft==0.13.0
!pip install -q datasets==2.19.0
!pip install -q "accelerate>=1.0.0"
!pip install -q sentencepiece==0.2.0
!pip install -q tqdm
!pip install -q huggingface-hub==0.26.0

print("✅ Core dependencies installed! If running in Colab, restart runtime after this cell.")

📦 Installing dependencies...
✅ Core dependencies installed! If running in Colab, restart runtime after this cell.


In [8]:
!pip install -q torchvision

## 💾 步骤2：挂载Google Drive（保存模型）

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# 创建输出目录
!mkdir -p /content/drive/MyDrive/gis-models
print("✅ Google Drive mounted!")

Mounted at /content/drive
✅ Google Drive mounted!


## 📂 步骤3：上传数据

**两种方式任选其一：**

### 方式A：从GitHub克隆仓库（推荐）

In [6]:
# 克隆仓库到当前目录
import os
import subprocess

print("📍 Current directory:", os.getcwd())

# 检查是否已克隆
if os.path.exists('/content/gis-code-ai'):
    print("✅ Repository already exists at /content/gis-code-ai")
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    print("✅ Repository already exists locally")
    os.chdir('gis-code-ai')
else:
    print("🔄 Cloning repository...")
    result = subprocess.run(
        ['git', 'clone', 'https://github.com/rockyistt/gis-code-ai.git'],
        cwd='/content',
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        os.chdir('/content/gis-code-ai')
        print("✅ Repository cloned successfully")
    else:
        print(f"❌ Clone failed: {result.stderr}")
        raise RuntimeError("Failed to clone repository")

print(f"📍 Working directory: {os.getcwd()}")

# 检查数据文件
print("\n📋 Checking data files...")
required_files = [
    'data/processed/file_level_instructions_weighted_variants_marked.jsonl',
    'data/processed/parsed_workflows.jsonl'
]

all_exist = True
for f in required_files:
    if os.path.exists(f):
        size_mb = os.path.getsize(f) / (1024*1024)
        print(f"✅ {f} ({size_mb:.1f} MB)")
    else:
        print(f"❌ {f}")
        all_exist = False

if all_exist:
    print("\n🎉 All data files ready! Ready to proceed with data preparation")
else:
    print("\n⚠️ Some data files are missing")

📍 Current directory: /content
✅ Repository already exists at /content/gis-code-ai
📍 Working directory: /content/gis-code-ai

📋 Checking data files...
✅ data/processed/file_level_instructions_weighted_variants_marked.jsonl (2.1 MB)
✅ data/processed/parsed_workflows.jsonl (13.8 MB)

🎉 All data files ready! Ready to proceed with data preparation


## 🔄 步骤4：准备训练数据

In [9]:
# 定义数据准备函数（直接在notebook中运行）
import json
from pathlib import Path
from tqdm import tqdm
import os

def prepare_file_level_training_data(instructions_file, workflows_file, output_dir, max_samples=None, split_ratio=0.9):
    """准备文件级训练数据"""

    # 检查文件是否存在
    if not os.path.exists(instructions_file):
        print(f"❌ Instructions file not found: {instructions_file}")
        print("\n📂 Available files in data/processed/:")
        if os.path.exists('data/processed'):
            for f in os.listdir('data/processed'):
                if os.path.isfile(f'data/processed/{f}'):
                    size = os.path.getsize(f'data/processed/{f}') / (1024*1024)
                    print(f"  - {f} ({size:.1f} MB)")
        else:
            print("  ❌ data/processed/ directory not found!")
        return None, None

    if not os.path.exists(workflows_file):
        print(f"❌ Workflows file not found: {workflows_file}")
        return None, None

    print(f"📖 Loading file-level instructions from {instructions_file}")
    instructions = {}
    with open(instructions_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                item = json.loads(line)
                file_id = item.get('file_id', '')
                instructions[file_id] = item
    print(f"✅ Loaded {len(instructions)} file-level instructions")

    print(f"📖 Loading workflows from {workflows_file}")
    workflows = {}
    with open(workflows_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                wf = json.loads(line)
                workflows[wf.get('file_id', '')] = wf
    print(f"✅ Loaded {len(workflows)} workflows")

    print("🔄 Converting to training format (file-level)...")
    training_samples = []

    count = 0
    for file_id, instr in tqdm(instructions.items()):
        workflow = workflows.get(file_id, {})

        if not workflow:
            continue

        # 清理指令
        instruction_text = instr.get('instruction', '').replace('**', '').replace('*', '')
        instruction_text = ' '.join(instruction_text.split()).strip()

        # 构建上下文
        test_app = workflow.get('test_app', '')
        database = workflow.get('database', '')
        total_steps = workflow.get('total_steps', 0)

        context_parts = []
        if test_app:
            context_parts.append(f"Application: {test_app}")
        if database:
            context_parts.append(f"Database: {database}")
        if total_steps > 0:
            context_parts.append(f"Steps: {total_steps}")
        input_context = " | ".join(context_parts)

        # 提取完整工作流JSON（所有steps）
        steps = workflow.get('steps', [])
        if not steps:
            continue

        workflow_output = {
            "workflow": {
                "metadata": {
                    "test_app": test_app,
                    "database": database,
                    "total_steps": len(steps)
                },
                "steps": steps
            }
        }
        output_code = json.dumps(workflow_output, indent=2, ensure_ascii=False)

        # 质量过滤
        if len(instruction_text.split()) < 5 or output_code == '{}':
            continue

        training_samples.append({
            "instruction": instruction_text,
            "input": input_context,
            "output": output_code
        })

        count += 1
        if max_samples and count >= max_samples:
            break

    print(f"✅ Created {len(training_samples)} training samples")

    if len(training_samples) == 0:
        print("❌ No training samples created! Check your data files.")
        return None, None

    # 划分训练集和验证集
    split_idx = int(len(training_samples) * split_ratio)
    train_data = training_samples[:split_idx]
    val_data = training_samples[split_idx:]

    print(f"📊 Split: {len(train_data)} train, {len(val_data)} validation")

    # 保存
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    train_file = output_dir / "training_data_train.json"
    val_file = output_dir / "training_data_val.json"

    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=2, ensure_ascii=False)
    print(f"💾 Train data saved: {train_file}")

    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=2, ensure_ascii=False)
    print(f"💾 Validation data saved: {val_file}")

    return train_data, val_data

print("✅ Data preparation function defined")

✅ Data preparation function defined


In [10]:
# 运行数据准备（直接调用函数）
import os

# 确保在正确的目录
if os.path.exists('/content/gis-code-ai'):
    os.chdir('/content/gis-code-ai')
elif os.path.exists('gis-code-ai'):
    os.chdir('gis-code-ai')

print("="*70)
print("🔄 开始数据准备...")
print(f"📍 Working directory: {os.getcwd()}")
print("="*70)

train_data, val_data = prepare_file_level_training_data(
    instructions_file='data/processed/file_level_instructions_weighted_variants_marked.jsonl',
    workflows_file='data/processed/parsed_workflows.jsonl',
    output_dir='data/training',
    max_samples=None,  # 使用全部数据，改为整数可限制样本数（如500）
    split_ratio=0.9
)

if train_data is not None:
    print("\n" + "="*70)
    print("🎉 数据准备完成！")
    print(f"📊 数据粒度: 文件级（完整工作流）")
    print(f"📊 训练样本: {len(train_data):,}")
    print(f"📊 验证样本: {len(val_data):,}")
    print("="*70)

    # 显示示例
    if train_data:
        print("\n📝 训练样本示例:")
        sample = train_data[0]
        print(f"Instruction: {sample['instruction'][:100]}...")
        print(f"Input: {sample['input']}")
        print(f"Output preview: {sample['output'][:200]}...")
else:
    print("\n❌ 数据准备失败！请检查上面的错误信息。")

🔄 开始数据准备...
📍 Working directory: /content/gis-code-ai
📖 Loading file-level instructions from data/processed/file_level_instructions_weighted_variants_marked.jsonl
✅ Loaded 1012 file-level instructions
📖 Loading workflows from data/processed/parsed_workflows.jsonl
✅ Loaded 1012 workflows
🔄 Converting to training format (file-level)...


100%|██████████| 1012/1012 [00:00<00:00, 1901.73it/s]

✅ Created 1012 training samples
📊 Split: 910 train, 102 validation
💾 Train data saved: data/training/training_data_train.json
💾 Validation data saved: data/training/training_data_val.json

🎉 数据准备完成！
📊 数据粒度: 文件级（完整工作流）
📊 训练样本: 910
📊 验证样本: 102

📝 训练样本示例:
Instruction: Workflow: create E MS Kabel, E HS Kabel, E LS Kabel in elektra in NRG Beheerkaart Elektra MS...
Input: Application: NRG Beheerkaart Elektra MS | Steps: 7
Output preview: {
  "workflow": {
    "metadata": {
      "test_app": "NRG Beheerkaart Elektra MS",
      "database": "",
      "total_steps": 7
    },
    "steps": [
      {
        "step_index": 0,
        "databas...


## 🚀 步骤5：开始训练

In [11]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset

print("🔧 Setting up training with CodeLlama...")

# ============================================================
# 配置参数 - CodeLlama (英语/荷兰语代码生成优化)
# ============================================================

MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
OUTPUT_DIR = "/content/drive/MyDrive/gis-models/codellama-gis-lora"  # 保存到Google Drive
TRAIN_FILE = "data/training/training_data_train.json"
VAL_FILE = "data/training/training_data_val.json"

# 训练参数（T4 GPU内存优化版 - 14GB显存限制）
NUM_EPOCHS = 3
BATCH_SIZE = 1  # Per-device batch size
GRADIENT_ACCUMULATION = 2  # Reduced from 4 → effective batch = 2
LEARNING_RATE = 2e-4  # CodeLlama推荐学习率
MAX_LENGTH = 512  # Reduced from 768 → significant memory savings

# LoRA参数（代码生成任务优化 - T4优化版）
LORA_R = 32  # Reduced from 64 → less trainable parameters
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

print(f"📦 Model: {MODEL_NAME}")
print(f"💾 Output: {OUTPUT_DIR}")
print(f"📊 Data: FILE-LEVEL (complete workflows)")
print(f"📈 Epochs: {NUM_EPOCHS}, Batch: {BATCH_SIZE}, Accumulation: {GRADIENT_ACCUMULATION}, LR: {LEARNING_RATE}")
print(f"📄 Max Length: {MAX_LENGTH}")
print(f"🎯 LoRA Rank: {LORA_R} (T4-optimized)")
print(f"🎯 Optimized for: T4 GPU (14GB VRAM) - memory-efficient config")

🔧 Setting up training with CodeLlama...
📦 Model: codellama/CodeLlama-7b-Instruct-hf
💾 Output: /content/drive/MyDrive/gis-models/codellama-gis-lora
📊 Data: FILE-LEVEL (complete workflows)
📈 Epochs: 3, Batch: 1, Accumulation: 2, LR: 0.0002
📄 Max Length: 512
🎯 LoRA Rank: 32 (T4-optimized)
🎯 Optimized for: T4 GPU (14GB VRAM) - memory-efficient config


In [12]:
# 加载tokenizer (CodeLlama)
print("📖 Loading CodeLlama tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Tokenizer loaded: vocab_size={len(tokenizer)}")

📖 Loading CodeLlama tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Tokenizer loaded: vocab_size=32016


In [13]:
# ⚠️ 必须在model loading前运行！升级关键库
print("🔧 Upgrading transformers, accelerate, and installing bitsandbytes...")

# 升级transformers到兼容版本
!pip install -q --upgrade transformers==4.46.0

# 升级accelerate到最新稳定版修复optimizer.train()错误
!pip install -q --upgrade accelerate>=1.0.0

# 安装bitsandbytes用于8-bit优化器（节省显存）
!pip install -q bitsandbytes

print("✅ Libraries upgraded!")
print(f"  transformers: 4.46.0")
print(f"  accelerate: >=1.0.0 (latest stable, fixes optimizer.train() bug)")
print(f"  bitsandbytes: installed (8-bit optimizer for memory efficiency)")

🔧 Upgrading transformers, accelerate, and installing bitsandbytes...
✅ Libraries upgraded!
  transformers: 4.46.0
  accelerate: >=1.0.0 (latest stable, fixes optimizer.train() bug)
  bitsandbytes: installed (8-bit optimizer for memory efficiency)
⚠️ IMPORTANT: Runtime restart recommended!
   Please go to: Runtime > Restart runtime
   Then re-run all cells in order


In [14]:
# 确保导入了必要的库
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

# 使用float16加载模型（不量化，稳定适配T4显存）
print("🤖 Loading CodeLlama-7B with float16 and device offloading...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

# 训练中必须禁用缓存以配合梯度检查点
model.config.use_cache = False

# 启用梯度检查点（节省显存）
model.gradient_checkpointing_enable()

print("✅ CodeLlama base model loaded (float16 with device offloading)")

# 应用LoRA
print("🔧 Applying LoRA...")
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA applied!")

🤖 Loading CodeLlama-7B with float16 and device offloading...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ CodeLlama base model loaded (float16 with device offloading)
🔧 Applying LoRA...
trainable params: 79,953,920 || all params: 6,818,500,608 || trainable%: 1.1726
✅ LoRA applied!


In [15]:
from datasets import load_dataset

# 准备数据集
print("📊 Preparing datasets...")

train_data = load_dataset('json', data_files=TRAIN_FILE, split='train')
eval_data = load_dataset('json', data_files=VAL_FILE, split='train')

print(f"  Train: {len(train_data)} samples")
print(f"  Val: {len(eval_data)} samples")

# 格式化prompt (CodeLlama优化格式)
def format_prompt(example):
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']

    # CodeLlama更适合直接的代码生成格式
    if input_text:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {input_text}

JSON Code:
{output}"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
{output}"""

    return {"text": prompt}

train_data = train_data.map(format_prompt, remove_columns=train_data.column_names)
eval_data = eval_data.map(format_prompt, remove_columns=eval_data.column_names)

# Tokenize
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 Tokenizing...")
train_dataset = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=train_data.column_names,
    desc="Tokenizing train"
)

eval_dataset = eval_data.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_data.column_names,
    desc="Tokenizing val"
)

print("✅ Datasets ready!")

📊 Preparing datasets...


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

  Train: 910 samples
  Val: 102 samples


Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

🔄 Tokenizing...


Tokenizing train:   0%|          | 0/910 [00:00<?, ? examples/s]

Tokenizing val:   0%|          | 0/102 [00:00<?, ? examples/s]

✅ Datasets ready!


In [16]:
# 配置训练
print("⚙️ Configuring training (T4-optimized)...")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_steps=50,  # Reduced from 100
    logging_steps=10,
    save_steps=300,  # Reduced from 500
    eval_steps=300,  # Reduced from 500
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    fp16=True,  # Enable fp16 compute
    bf16=False,
    optim="adamw_8bit",  # 8-bit AdamW for memory efficiency (requires bitsandbytes)
    lr_scheduler_type="cosine",
    save_total_limit=2,  # Reduced from 3 to save disk space
    report_to="none",
    logging_dir=f"{OUTPUT_DIR}/logs",
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    push_to_hub=False,
    gradient_checkpointing=True,  # Memory savings via checkpointing
    max_grad_norm=1.0,  # Gradient clipping
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,  # Use processing_class instead of tokenizer (new API)
    data_collator=data_collator,
)

print("✅ Trainer ready!")
print("\n" + "="*70)
print("🚀 Starting training (T4-optimized, may take 1-2 hours)...")
print("="*70)

# 训练前清理显存，减少碎片
import gc
gc.collect()
torch.cuda.empty_cache()

# 开始训练
trainer.train()

print("\n" + "="*70)
print("🎉 Training completed!")
print("="*70)

⚙️ Configuring training (T4-optimized)...
✅ Trainer ready!

🚀 Starting training (T4-optimized, may take 6-8 hours)...


Step,Training Loss,Validation Loss
300,0.050500,0.047387
600,0.043000,0.044602
900,0.047200,0.044234
1200,0.037500,0.043698



🎉 Training completed!


In [17]:
# 保存模型
print("💾 Saving model...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Model saved to {OUTPUT_DIR}")

# 保存训练信息
import json
training_info = {
    "model_name": MODEL_NAME,
    "num_epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "train_samples": len(train_dataset),
    "val_samples": len(eval_dataset),
}

with open(f"{OUTPUT_DIR}/training_info.json", 'w') as f:
    json.dump(training_info, f, indent=2)

print("\n📊 Training Summary:")
for key, value in training_info.items():
    print(f"  {key}: {value}")

💾 Saving model...
✅ Model saved to /content/drive/MyDrive/gis-models/codellama-gis-lora

📊 Training Summary:
  model_name: codellama/CodeLlama-7b-Instruct-hf
  num_epochs: 3
  batch_size: 1
  learning_rate: 0.0002
  lora_r: 32
  lora_alpha: 16
  train_samples: 910
  val_samples: 102


## 🧪 步骤6：测试模型

In [18]:
# 快速测试 (CodeLlama)
print("🧪 Testing CodeLlama model inference...")

test_instruction = "Create a new MS cable object at coordinates (186355533, 439556907)"
test_context = "Application: PowerGrid | Database: ND | Steps: 5"

prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {test_instruction}
Context: {test_context}

JSON Code:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

print("\n🔮 Generating...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("JSON Code:")[-1].strip()

print("\n" + "="*70)
print("📝 Test Result:")
print("="*70)
print(f"Instruction: {test_instruction}")
print(f"Context: {test_context}")
print(f"\nGenerated Output:\n{response[:500]}...")
print("="*70)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🧪 Testing CodeLlama model inference...

🔮 Generating...

📝 Test Result:
Instruction: Create a new MS cable object at coordinates (186355533, 439556907)
Context: Application: PowerGrid | Database: ND | Steps: 5

Generated Output:
{
  "workflow": {
    "metadata": {
      "test_app": "PowerGrid",
      "database": "ND",
      "total_steps": 5
    },
    "steps": [
      {
        "step_index": 0,
        "database": ":catalog",
        "object": "Object Control",
        "object_id": "Passed",
        "module": "Tabs",
        "method": "Select Tab",
        "command": "Execute Tabs Testcommand",
        "test_data": {
          "create": {},
          "update": {},
          "editor": {}
        }
      },
      {
      ...


## 📦 步骤7：下载模型（可选）

In [ ]:
# 打包模型用于下载
!cd /content/drive/MyDrive/gis-models && zip -r codellama-gis-lora.zip codellama-gis-lora/
print("✅ Model zipped!")
print(f"📦 Location: /content/drive/MyDrive/gis-models/codellama-gis-lora.zip")
print("💡 You can download it from Google Drive")

In [ ]:
# 清理并重新保存模型（确保adapter_model.bin正确生成）
import os
import shutil
from pathlib import Path

OUTPUT_DIR = "/content/drive/MyDrive/gis-models/codellama-gis-lora"

print("🔧 清理并重新保存模型...")
print("=" * 70)

# 1. 清理旧的不完整文件
print("\n1️⃣ 清理旧文件...")
if os.path.exists(OUTPUT_DIR):
    # 删除可能不完整的文件
    files_to_remove = [
        "adapter_model.bin",  # 如果存在但损坏
        "training_args.bin",
        "optimizer.pt"
    ]
    for f in files_to_remove:
        fpath = os.path.join(OUTPUT_DIR, f)
        if os.path.exists(fpath):
            os.remove(fpath)
            print(f"   ❌ 已删除: {f}")
else:
    print(f"   📁 创建目录: {OUTPUT_DIR}")
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. 保存LoRA模型
print("\n2️⃣ 保存LoRA权重...")
try:
    model.save_pretrained(OUTPUT_DIR)
    print(f"   ✅ 保存完成: {OUTPUT_DIR}")
except Exception as e:
    print(f"   ❌ 保存失败: {e}")
    raise

# 3. 保存Tokenizer
print("\n3️⃣ 保存Tokenizer...")
try:
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"   ✅ Tokenizer保存完成")
except Exception as e:
    print(f"   ❌ Tokenizer保存失败: {e}")
    raise

# 4. 验证所有必要文件
print("\n4️⃣ 验证文件...")
required_files = [
    "adapter_config.json",
    "adapter_model.bin",      # ⭐ 关键文件
    "tokenizer.model",
    "special_tokens_map.json",
    "tokenizer_config.json"
]

all_files_present = True
for f in required_files:
    fpath = os.path.join(OUTPUT_DIR, f)
    if os.path.exists(fpath):
        size_mb = os.path.getsize(fpath) / (1024*1024)
        print(f"   ✅ {f} ({size_mb:.1f}MB)")
    else:
        print(f"   ❌ {f} (缺失!)")
        all_files_present = False

# 5. 保存训练信息
print("\n5️⃣ 保存训练信息...")
import json
training_info = {
    "model_name": "codellama/CodeLlama-7b-Instruct-hf",
    "num_epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": LEARNING_RATE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "lora_dropout": LORA_DROPOUT,
}

with open(os.path.join(OUTPUT_DIR, "training_info.json"), 'w') as f:
    json.dump(training_info, f, indent=2)

print("=" * 70)
if all_files_present and os.path.getsize(os.path.join(OUTPUT_DIR, "adapter_model.bin")) > 100*1024*1024:
    print("✅ 模型保存完成！adapter_model.bin已正确生成")
    print(f"💾 模型位置: {OUTPUT_DIR}")
else:
    print("⚠️ 警告：adapter_model.bin可能未完整保存")
    print("   建议：检查Google Drive空间是否足够（需要~6GB）")

## 🎯 下一步

训练完成后，你可以：

1. **在Colab中继续测试**：使用上面的测试单元格
2. **下载模型**：从Google Drive下载打包的模型
3. **本地部署**：将模型下载到本地进行推理
4. **评估模型**：运行完整的评估脚本

### 本地使用模型

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 加载CodeLlama基座模型
base_model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
model = PeftModel.from_pretrained(base_model, "path/to/codellama-gis-lora")
tokenizer = AutoTokenizer.from_pretrained("path/to/codellama-gis-lora")

# 推理
# ...
```

---

**训练参数调优建议：**
- T4 GPU: batch_size=4, gradient_accumulation=4
- A100 GPU: batch_size=8-12, gradient_accumulation=2-4
- 如果显存不足：减小batch_size或max_length
- 训练时间估计：T4约4-6小时，A100约1-2小时

**🎉 恭喜！你已经完成了GIS代码生成模型的训练（CodeLlama版）！**

## 🧪 步骤8: 加载模型和评估

**已完成模型训练？现在让我们加载它并在测试集上评估性能！**

两种方式选择:
1. **快速推理测试** (下面单元格)
2. **完整评估** (后续单元格)


In [5]:
# 检查Google Drive和模型文件
import os
from pathlib import Path

print("🔍 诊断：检查Google Drive和模型文件")
print("=" * 70)

# 1. 检查Google Drive是否挂载
drive_path = "/content/drive/MyDrive"
if os.path.exists(drive_path):
    print(f"✅ Google Drive已挂载")
    print(f"   路径: {drive_path}")
else:
    print(f"❌ Google Drive未挂载")
    print(f"\n解决方案:")
    print(f"请运行:")
    print(f"```python")
    print(f"from google.colab import drive")
    print(f"drive.mount('/content/drive')")
    print(f"```")
    raise RuntimeError("Google Drive未挂载，请先挂载")

# 2. 检查gis-models目录
gis_models_path = "/content/drive/MyDrive/gis-models"
if os.path.exists(gis_models_path):
    print(f"\n✅ /gis-models/ 目录存在")
    print(f"   路径: {gis_models_path}")

    # 列出目录内容
    print(f"\n   📂 目录内容:")
    try:
        items = os.listdir(gis_models_path)
        for item in items:
            item_path = os.path.join(gis_models_path, item)
            if os.path.isdir(item_path):
                print(f"      📁 {item}/ (目录)")
            else:
                size_mb = os.path.getsize(item_path) / (1024*1024)
                print(f"      📄 {item} ({size_mb:.1f}MB)")
    except Exception as e:
        print(f"      ❌ 无法列出目录: {e}")
else:
    print(f"❌ /gis-models/ 目录不存在")
    print(f"   需要创建: {gis_models_path}")
    os.makedirs(gis_models_path, exist_ok=True)
    print(f"✅ 已创建目录")

# 3. 检查CodeLlama模型文件
model_path = "/content/drive/MyDrive/gis-models/codellama-gis-lora"
if os.path.exists(model_path):
    print(f"\n✅ CodeLlama模型目录存在")
    print(f"   路径: {model_path}")

    # 列出必要文件
    required_files = [
        "adapter_config.json",
        "adapter_model.bin",
        "tokenizer.model",
        "special_tokens_map.json",
        "tokenizer_config.json"
    ]

    print(f"\n   📂 模型文件检查:")
    missing_files = []
    for f in required_files:
        file_path = os.path.join(model_path, f)
        if os.path.exists(file_path):
            size_mb = os.path.getsize(file_path) / (1024*1024)
            print(f"      ✅ {f} ({size_mb:.1f}MB)")
        else:
            print(f"      ⚠️ {f} (未找到)")
            missing_files.append(f)

    if missing_files:
        print(f"\n   ⚠️ 缺失的关键文件可能影响模型加载")
else:
    print(f"\n❌ CodeLlama模型目录不存在")
    print(f"   期望位置: {model_path}")
    print(f"\n📋 可能的原因:")
    print(f"   1. 模型还没有训练完成")
    print(f"   2. 模型保存到了不同的位置")
    print(f"   3. 模型文件上传失败")
    print(f"\n💡 解决方案:")
    print(f"   选项A: 检查训练是否完成（查看步骤5-7的输出）")
    print(f"   选项B: 检查Google Drive中的实际位置")
    print(f"   选项C: 重新运行训练步骤（步骤5-7）")
    raise FileNotFoundError(f"模型文件未找到: {model_path}")

print("\n" + "=" * 70)
print("✅ 所有检查通过！可以继续加载模型")
print("=" * 70)

🔍 诊断：检查Google Drive和模型文件
✅ Google Drive已挂载
   路径: /content/drive/MyDrive

✅ /gis-models/ 目录存在
   路径: /content/drive/MyDrive/gis-models

   📂 目录内容:
      📁 codellama-gis-lora/ (目录)

✅ CodeLlama模型目录存在
   路径: /content/drive/MyDrive/gis-models/codellama-gis-lora

   📂 模型文件检查:
      ✅ adapter_config.json (0.0MB)
      ⚠️ adapter_model.bin (未找到)
      ✅ tokenizer.model (0.5MB)
      ✅ special_tokens_map.json (0.0MB)
      ✅ tokenizer_config.json (0.0MB)

   ⚠️ 缺失的关键文件可能影响模型加载

✅ 所有检查通过！可以继续加载模型


In [12]:
# 从Google Drive加载已训练的模型
print("📦 加载模型...")
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from pathlib import Path
import torch

MODEL_PATH = "/content/drive/MyDrive/gis-models/codellama-gis-lora"

print(f"📍 模型路径: {MODEL_PATH}")

# 验证路径存在
if not Path(MODEL_PATH).exists():
    raise FileNotFoundError(f"❌ 模型路径不存在: {MODEL_PATH}")

# 加载Tokenizer (使用local_files_only避免HF验证错误)
print("\n📖 加载Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    padding_side="right",
    local_files_only=True,  # 关键：告诉HF这是本地路径
    trust_remote_code=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer加载完成")

# 加载基础模型
print("\n🤖 加载基础模型...")
base_model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
base_model.config.use_cache = False
print("✅ 基础模型加载完成")

# 加载LoRA权重 (同样使用local_files_only)
print("\n🔧 加载LoRA权重...")
model = PeftModel.from_pretrained(
    base_model,
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    local_files_only=True,  # 关键：告诉PEFT这是本地路径
)
model.eval()
print("✅ 模型加载完成！\n")

📦 加载模型...
📍 模型路径: /content/drive/MyDrive/gis-models/codellama-gis-lora

📖 加载Tokenizer...
✅ Tokenizer加载完成

🤖 加载基础模型...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ 基础模型加载完成

🔧 加载LoRA权重...
✅ 模型加载完成！



In [13]:
# 快速推理测试
print("=" * 70)
print("🧪 快速推理测试")
print("=" * 70)

def generate_code(instruction, context=""):
    """使用加载的模型生成代码"""
    if context:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}
Context: {context}

JSON Code:
"""
    else:
        prompt = f"""You are a GIS workflow code generator. Generate complete JSON workflow code based on the instruction.

Instruction: {instruction}

JSON Code:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    code = text.split("JSON Code:")[-1].strip()
    return code

# 测试案例
test_cases = [
    {
        "instruction": "Create a new MS cable object at coordinates (186355533, 439556907)",
        "context": "Application: PowerGrid | Database: ND | Steps: 5"
    },
    {
        "instruction": "Open object in editor and verify fields",
        "context": "Application: NRG Elektra | Database: elektra | Steps: 3"
    }
]

for i, test in enumerate(test_cases, 1):
    print(f"\n📝 测试案例 {i}:")
    print(f"  指令: {test['instruction']}")
    print(f"  上下文: {test['context']}")

    result = generate_code(test['instruction'], test['context'])

    # 检查JSON有效性
    try:
        json.loads(result)
        print(f"  ✅ JSON有效")
    except:
        print(f"  ⚠️ JSON格式问题")

    print(f"  💻 生成代码预览 (前200字符):\n  {result[:200]}...")

🧪 快速推理测试

📝 测试案例 1:
  指令: Create a new MS cable object at coordinates (186355533, 439556907)
  上下文: Application: PowerGrid | Database: ND | Steps: 5
  ⚠️ JSON格式问题
  💻 生成代码预览 (前200字符):
  {
  "workflow": {
    "metadata": {
      "test_app": "PowerGrid",
      "database": "ND",
      "total_steps": 5
    },
    "steps": [
      {
        "step_index": 0,
        "database": ":catalog",...

📝 测试案例 2:
  指令: Open object in editor and verify fields
  上下文: Application: NRG Elektra | Database: elektra | Steps: 3
  ⚠️ JSON格式问题
  💻 生成代码预览 (前200字符):
  {
  "workflow": {
    "metadata": {
      "test_app": "NRG Elektra",
      "database": "elektra",
      "total_steps": 3
    },
    "steps": [
      {
        "step_index": 0,
        "database": ":el...
